In [33]:
#Import and set magics
import numpy as np
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader
import pydst
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import scipy as sp
from scipy import stats
from scipy import optimize
from scipy import interpolate
from scipy import linalg
import sympy as sm

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
%matplotlib inline
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot


# Estimating income processes


Consider $N$ households indexed by $i$ who are in the labor market for $T$ periods indexed by $t \in ${1,2,...,T}$,

Their **wage income** is stochastic and given by,
 
$P_{i,t}$ = $\psi_{i,t}$ $P_{i,t-1}$

$\tilde{Y}_{i,t}$ = $xi_{i,t}$ $P_{i,t}$

$Y_{i,t}$ = \left\begin{array}{rcl}$
            \overline{\overline{0}+\text{if}\mu_{i,t}<\pi}
            \\tilde{Y}_{i,t} +\text{else}
            \end{array}\right.


   


$\psi_{i,t}$ ~ LogNormal(-0.5$\sigma_{\psi}^2$,$\sigma_{\psi})$

$\xi_{i,t}$ ~ LogNormal(-0.5$\sigma_{\xi}^2$,$\sigma_{\xi})$

$\mu_{i,t}$ ~ $Uniform(0,1)$


$P_{0} = 1$
   
where

$\sigma_{\psi}$ is the standard deviation of the *permanent* shocks, $\psi_{i,t}$

$\sigma_{\xi}$ is the standard deviation of the *transitory* shocks, $\xi_{i,t}$

$\pi$ is the risk of unemployment

The data you have access to is:


In [87]:
import numpy as np
dataY = np.load(r'C:\Users\musse\Documents\GitHub\projects-2020-zjr-hrv\projects-2020-zjr-hrv\dataY.npy')
T,N = dataY.shape
print(dataY)
print(dataY.shape)




[[0.83317569 0.72881172 0.         ... 1.16771806 0.93207083 0.86711803]
 [1.18542294 0.92826337 1.62913142 ... 1.13903869 0.94479246 0.78842682]
 [1.14813596 0.90542496 0.70634401 ... 1.49584791 1.08969956 0.        ]
 ...
 [0.73818651 0.59958039 0.56135238 ... 2.60075173 1.07070489 0.43010036]
 [1.14130879 0.85728537 0.54530761 ... 3.79294856 0.67764143 0.38720822]
 [0.64908127 0.85101393 0.59517395 ... 3.32800991 0.82400879 0.5814573 ]]
(20, 50000)


**Question 1:** Calculate income growth rates as log-changes

In [102]:
#Defining income growth rate by finding the first differences. Thereafter replacing the negative Y-values with .nan

logY = np.diff(np.log(dataY))
#diff_dataY = np.diff(dataY) 
  
# printing initial arrays 
print("Initial diff", logY) 
  
# code to replace all negative value with 0 
logY[logY<=0] = np.nan
  
# printing result 
print("New resulting diff: ", logY) 






initial diff [[-0.13382911        -inf         inf ...  0.45991757 -0.22539794
  -0.0722337 ]
 [-0.2445394   0.56248678 -0.45247293 ...  0.10650339 -0.18697465
  -0.18092569]
 [-0.2374906  -0.24830202  0.49367077 ...  0.38133059 -0.31679118
         -inf]
 ...
 [-0.20796646 -0.06588123  0.6550227  ...  1.08826931 -0.88748332
  -0.9120539 ]
 [-0.2861601  -0.45242079  0.43603115 ...  1.35354726 -1.7222807
  -0.5596557 ]
 [ 0.27087056 -0.35757477  0.31896575 ...  1.52939372 -1.39594858
  -0.34864367]]
New resulting diff:  [[       nan        nan        inf ... 0.45991757        nan        nan]
 [       nan 0.56248678        nan ... 0.10650339        nan        nan]
 [       nan        nan 0.49367077 ... 0.38133059        nan        nan]
 ...
 [       nan        nan 0.6550227  ... 1.08826931        nan        nan]
 [       nan        nan 0.43603115 ... 1.35354726        nan        nan]
 [0.27087056        nan 0.31896575 ... 1.52939372        nan        nan]]


**Question 2:** Calculate the following 3 statistics from the data:

$s_{1}$: Share of observations with $Y_{i,t}$=0

$s_{2}$: Variance of income growth rate, Var($\Delta$ Log $Y_{i,t}$)

$s_{3}$: Covariance of income growth rates one period apart, Cov($\Delta$ Log $Y_{i,t}$,$\Delta$ Log $Y_{i,t-1}$ )

To count shares of observations with $Y_{i,t}$=0, we use the nonzero-function

In [103]:
#S1:
print('Number of observations with Y=0 is') 
np.count_nonzero(dataY==0)



Number of observations with Y=0 is


49925

The variance is calculated by the mean of square minus the square of the mean, i.e.

$Var(x)=E[(X-\mu)^{2}]$



In [104]:
#S2:
print(np.var(logY))

nan


As seen above, the variance results in "nan", meaning we would have to remove the nan values from our sample to find a value for the variance. Hence, we find:

In [106]:
diffY = diff_dataY[~np.isnan(diff_dataY)]
print('The variance is' ,np.var(diffY))

The variance is 0.15780801990005514


Finally, the Covariance is found by

In [94]:
np.cov(diffY)

array(0.15780833)

Question 3: Simulate the income process using your own choice of $\sigma_{\psi}$, $\sigma_{\xi}$, $\pi$, T and N. Calculate the 3 same statistics. Compare with the data statistics

In [119]:
#Simulated figures
T = 50
N = 1000
pi = 0.3
s_psi = 0.23
s_xi = 0.4

#Model
psi = np.random.lognormal(-0.5 * s_psi, s_psi, 1000)
xi = np.random.lognormal(-0.5 * s_xi,s_xi, 1000)
P_0 = 1
P = psi * P_0
Y_tilde = xi * P
mu = np.random.uniform(0,1)


#Defining Y
def income(Y): 
    if mu < pi:
        Y=0 
    else :
        Y = Y_tilde

In [ ]:
import scipy as sp
import random
g = random.Random()
g.seed(1234)

import random, math
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import SimPy.Simulation as Sim

initialize(): set the simulation time and the event list
• activate(): used to mark a thread (process) as runnable when it is first
created and add its first event into the event list.
• simulate(): starts the simulation

**Question 4:** Solve the following minimization problem to estimate $\sigma_{\psi}$, $\sigma_{\xi}$ and $\pi$ 

$\sigma_{\psi}^*$, $\sigma_{\xi}^*$, $\pi^*$ = $arg_{\sigma_{\psi}>=0,\sigma_{\xi}>=0,\pi\in [0,1]}$               min $(s_{1}^{sim}-s_{1}^{data})^2$ + $(s_{2}^{sim}-s_{2}^{data})^2$ + $(s_{3}^{sim}-s_{3}^{data})^2$

where for each new guess of $\sigma_{\psi}$, $\sigma_{\xi}$ and $\pi$ you should be re-simulating the data with the same seed and re-calculate the 3 statistics

In [ ]:
def objective():
 pass
res = optimize.minimize(objective,x,method='L-BFGS-B',bounds=?,args=?,options={'eps': 1e-4})
# hint: options={'eps':1e-4} uses a larger step-size when approximating the jacbian, which is useful in this case


# Durable purchases

Consider a **household** living in two periods.

In the **second period** it gets utility from **non-durable consumption**, $c$, and **durable consumption**, $d+\chi x$:

$$
\begin{aligned}
v_{2}(m_{2},d)&= \max_{c}\frac{(c^{\alpha}(d+\chi x)^{1-\alpha})^{1-\rho}}{1-\rho}\\
\text{s.t.} \\
x &= m_{2}-c \\
c &\in [0,m_{2}]
\end{aligned}
$$

where 

* $m_2$ is cash-on-hand in the beginning of period 2
* $c$ is non-durable consumption
* $d$ is pre-commited durable consumption
* $x = m_2 - c$ is extra durable consumption
* $\rho > 1$ is the risk aversion coefficient
* $\alpha \in (0,1)$ is the utility weight on non-durable consumption
* $\chi \in (0,1)$ implies that extra durable consumption is *less* valuable than pre-comitted durable consumption
* the second constraint ensures the household *cannot* die in debt

The **value function** $v_2(m_2,d)$ measures the household's value of having $m_2$ at the beginning of period 2 with precomitted durable consumption of $d$. The optimal choice of non-durable consumption is denoted $c^{\ast}(m_2,d)$. The optimal extra durable consumption function is $x^{\ast}(m_2,d) = m_2-c^{\ast}(m_2,d)$.

Define the so-called **end-of-period 1 value function** as:

$$
\begin{aligned}
w(a,d)&\equiv\beta\mathbb{E}_{1}\left[v_2(m_2,d)\right]
\end{aligned}
$$

where 

$$
\begin{aligned}
m_2&= (1+r)a+y \\
y &= \begin{cases}
1-\Delta & \text{with prob. }\frac{1}{3}\\
1 & \text{with prob. }\frac{1}{3}\\
1+\Delta & \text{with prob. }\frac{1}{3}
\end{cases}\\
\end{aligned}
$$

and

* $a$ is assets at the end of period 1
* $\beta > 0$ is the discount factor
* $\mathbb{E}_1$ is the expectation operator conditional on information in period 1
* $y$ is income in period 2
* $\Delta \in (0,1)$ is the level of income risk (mean-preserving)
* $r$ is the return on savings

In the **first period**, the household chooses it's pre-comitted level of durable consumption for the next-period,

$$
\begin{aligned}
v_{1}(m_{1})&=\max_{d} w(a,d)\\&\text{s.t.}&\\
a&= m_{1}-d \\
d&\in [0,m_{1}]\\
\end{aligned}
$$

where $m_1$ is cash-on-hand in period 1. The second constraint ensures the household *cannot* borrow. The **value function** $v_1(m_1)$ measures the household's value of having $m_1$ at the beginning of period 1. The optimal choice of pre-committed durable consumption is denoted $d^{\ast}(m_1)$.

The **parameters** and **grids** for $m_1$, $m_2$ and $d$ should be:

In [3]:
# a. parameters
rho = 2
alpha = 0.8
beta = 0.96
chi = 0.9
r = 0.04
Delta = 0.25

# b. grids
m1_vec = np.linspace(1e-8,10,100)
m2_vec = np.linspace(1e-8,10,100)
d_vec = np.linspace(1e-8,5,100)




# Refined grid search

Let $\boldsymbol{x} = \left[\begin{array}{c}
x_1 \\
x_2\\
\end{array}\right]$ be a two-dimensional vector. Consider the following algorithm:

**Algorithm:** `grid_search()`

**Goal:** Minimize the function $f(\boldsymbol{x})$.

1. Choose a grid size $N$ and minimum and maximum values of $x_{1}$ and $x_{2}$ denoted $\overline{x_{1}}$>$\underline{x_{1}}$ and $\overline{x_{2}}$>$\underline{x_{2}}$
2. Calculate step sizes 

    $\Delta_{1}$ =($\overline{x_{1}}$-$\underline{x_{1}}$)/(N-1)
    
    $\Delta_{2}$ =($\overline{x_{2}}$-$\underline{x_{2}}$)/(N-1)
3. Find the grid point with the lowest function value solving 

    $j^{*}_{1}$, $j^{*}_{2}$ = $arg_{j_{1}\in{0,...,N-1}, j_{2}\in{0,...,N-1}}$  f( $\underline{x_{1}+j_{1}$\Delta_{1}$,$\underline{x_{2}+j_{2}$\Delta_{2}$)

    
    


1. Return 


**Question 1:** Implement the grid_search() algorithm to minimize the rosen function.


In [45]:
def rosen(x):
 return (1.0-x[0])**2+2*(x[1]-x[0]**2)**2


In [46]:
def grid_search(f,x1_min,x1_max,x2_min,x2_max,N):
 return np.nan,np.nan,np.nan
# settings
x1_min = 0
x1_max = 5
x2_min = 0
x2_max = 4
N = 1000
# apply grid search
x1,x2,f = grid_search(rosen,x1_min,x1_max,x2_min,x2_max,N)
print(f'minimum found at ({x1:.8f},{x2:.8f}) with the function value {f:.8f}')

minimum found at (nan,nan) with the function value nan


**Question 2:** Implement the refined_grid_search() algorithm to minimize the rosen function

In [47]:
def refined_grid_search(f,x1_min,x1_max,x2_min,x2_max,N,K):
 return np.nan,np.nan,np.nan
# more settings
K = 10
# apply refined grid search
x1,x2,f = refined_grid_search(rosen,x1_min,x1_max,x2_min,x2_max,N,K)
print(f'minimum found at ({x1:.8f},{x2:.8f}) with the function value {f:.8f}'

SyntaxError: unexpected EOF while parsing (<ipython-input-47-36645f060032>, line 7)